# **PROYECTO INDIVIDUAL STEAM**

Este cuaderno Jupyter marca el inicio de mi proyecto como MLOps Engineer en Steam Games, centrado en la implementación de una fase ETL (Extracción, Transformación y Carga) integral y avanzada. Mi objetivo es estructurar y refinar nuestros datos de manera eficiente, estableciendo así una base sólida para el desarrollo de endpoints sofisticados. Estos endpoints serán cruciales para alimentar nuestras funciones avanzadas y el modelo de recomendación, impulsando nuestra capacidad de ofrecer experiencias de juego personalizadas y optimizadas. A través de este proceso, aplicaré las mejores prácticas de ingeniería de datos y machine learning operations, garantizando que nuestra arquitectura de datos sea escalable, confiable y segura, y que esté alineada con las últimas tendencias y necesidades del mercado de juegos

In [1]:
#importar bibliotecas
import pandas as pd
import numpy as np
import ast
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import re
from datetime import datetime

## **ETL DE STEAM GAMES** 

**Extracción de Datos**

In [50]:
# Carga de datos de juegos de Steam desde un archivo JSON comprimido
steam_games = pd.read_json('data/steam_games.json.gz', compression='gzip', lines=True)


**Evaluación de datos**
Revisaremos los datos sin transformación, tal como se solicita y eliminares las columnas que sean innecesarias para mejor perfomance de las consultas y modelo de machine learning.

In [51]:
# Visualización de información general del DataFrame cargado
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [52]:
#Revisamos las primeras 5 columnas
steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Podemos observar que hay varias filas con valores nulos, así que debemos considerar eliminarlas

In [53]:
#Revisamos las últimas 5 columnas
steam_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [54]:
# Eliminar filas donde todos los elementos son NaN
steam_games.dropna(how='all', inplace=True)

**Revisaremos si hay listas anidadas ya que esto es algo usual cuando se guarda en formato JSON y luego analizaremos si es necesario desanidarlas**

In [55]:
datos_columnas = {"columna":[], "tipos_de_datos":[]} # Creo un diccionario para recolectar información de las columnas

for columna in steam_games.columns: # Itero sobre cada columna del DataFrame steam_games
    datos_columnas["columna"].append(columna) # Agrego el nombre de la columna actual al diccionario
    # Obtengo y almaceno los tipos de datos únicos en la columna actual
    datos_columnas["tipos_de_datos"].append(steam_games[columna].apply(type).unique())

analisis = pd.DataFrame(datos_columnas) # Construyo un DataFrame con la información recopilada
analisis # Visualizo el DataFrame analisis

,columna,tipos_de_datos
0,publisher,"[<class 'str'>, <class 'NoneType'>]"
1,genres,"[<class 'list'>, <class 'NoneType'>]"
2,app_name,"[<class 'str'>, <class 'NoneType'>]"
3,title,"[<class 'str'>, <class 'NoneType'>]"
4,url,[<class 'str'>]
5,release_date,"[<class 'str'>, <class 'NoneType'>]"
6,tags,"[<class 'list'>, <class 'NoneType'>]"
7,reviews_url,"[<class 'str'>, <class 'NoneType'>]"
8,specs,"[<class 'list'>, <class 'NoneType'>]"
9,price,"[<class 'float'>, <class 'str'>, <class 'NoneT..."


Las columnas que contienen 'list' aluden a datos con estructura anidada que requieren un manejo especial.

**Vamos a preparar a la columna "id" para para poder usarla más adelante sin ningún inconveniente.**

In [56]:
# Verificar que todos los 'id' flotantes no tienen una parte decimal significativa
# Redondear y comparar con el valor original para asegurarte de que son enteros
ids_are_integers = (steam_games['id'].fillna(0).apply(float.is_integer)).all()
if ids_are_integers:
    # Si todos los 'id' pueden ser representados como enteros, realiza la conversión
    steam_games['id'] = steam_games['id'].astype('Int64')  # Usamos 'Int64' (con I mayúscula) para permitir valores nulos
else:
    # Maneja el caso de que algunos 'id' no sean enteros, por ejemplo, con un error o un procedimiento diferente
    raise ValueError("Not all 'id' values are integers.")

# Renombrar la columna 'id' a 'item_id'
steam_games.rename(columns={'id': 'item_id'}, inplace=True)

In [57]:
#Revisamos el cambio
steam_games.dtypes['item_id']

Int64Dtype()

**Creamos la columna "year_of_release" para extraer el año:**

In [58]:
# Extraemos el año considerando que tiene un formato 'YYYY-...'
#steam_games['year_of_release'] = steam_games['release_date'].str[:4]#
steam_games['year_of_release'] = steam_games['release_date'].str.extract(r'(\d{4})').fillna('0')
steam_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year_of_release
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290,Poolians.com,2017


In [59]:
steam_games.isnull().sum()

publisher          8052
genres             3283
app_name              2
title              2050
url                   0
release_date       2067
tags                163
reviews_url           2
specs               670
price              1377
early_access          0
item_id               2
developer          3299
year_of_release       0
dtype: int64

In [60]:
#Como app_name tiene menos nulos que title, trabajaremos con app_name.
#Revisemos los nuños de appname para ver si la información es relevante
filas_con_nulos_app_name = steam_games[steam_games['app_name'].isnull()]

filas_con_nulos_app_name

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year_of_release
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,<NA>,None,0
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160,None,2014


Observamos que la primera entrada carece de información relevante, lo que indica que debería ser eliminada. En cuanto a la segunda entrada, tras una revisión en la web, se identificó que corresponde al juego 'Duet'. Por tanto, procederemos a actualizar esta entrada con el nombre del juego y asignaremos 'Kumobius' tanto en los campos de publisher como de desarrollador.

In [61]:
# Eliminar la fila con index 88384
steam_games.drop(index=88384, inplace=True)

# Actualizar la fila con index 90890
steam_games.loc[90890, 'app_name'] = 'Duet'
steam_games.loc[90890, 'publisher'] = 'Kumobius'
steam_games.loc[90890, 'developer'] = 'Kumobius'

# Mostrar la fila actualizada para verificar los cambios
steam_games.loc[90890]

publisher                                                   Kumobius
genres                                               [Action, Indie]
app_name                                                        Duet
title                                                           None
url                      http://store.steampowered.com/app/317160/_/
release_date                                              2014-08-26
tags                                                 [Action, Indie]
reviews_url        http://steamcommunity.com/app/317160/reviews/?...
specs                                     [Single-player, Game demo]
price                                                           None
early_access                                                     0.0
item_id                                                       317160
developer                                                   Kumobius
year_of_release                                                 2014
Name: 90890, dtype: object

In [62]:
steam_games. head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year_of_release
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290,Poolians.com,2017


**Descomposición de la Columna 'genres' en el DataFrame de Steam Games**
A continuación, transformaremos la columna 'genres' de nuestro DataFrame steam_games. El objetivo es expandir las listas de géneros contenidas en esta columna para que cada género esté en su propia fila. Esto nos permitirá analizar los géneros de manera más granular. Posteriormente, eliminaremos cualquier fila que haya quedado con un valor faltante en 'genres' tras la expansión.

Este proceso se compone de dos pasos principales:

* Utilizamos el método explode para desglosar cada elemento de las listas en la columna 'genres' en filas separadas.
* Limpiamos el DataFrame resultante eliminando las filas donde la columna 'genres' es nula, lo cual podría afectar nuestros análisis posteriores.

In [63]:
# Expandir cada elemento en la columna 'genres' a su propia fila
steam_games = steam_games.explode('genres')

# Mostrar las nuevas dimensiones del DataFrame para confirmar los cambios
shape_after_explosion = steam_games.shape
shape_after_explosion

(74836, 14)

**Revisemos los cambios**

In [64]:
steam_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year_of_release
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018


**Eliminamos las columnas que son innecesarias para el sistema de recomendación de videojuegos y las funciones**

In [65]:
# Seleccionar columnas relevantes para el análisis final
steam_games = steam_games[['item_id', 'app_name', 'genres', 'year_of_release', 'publisher', 'developer']]
# Visualizar las primeras dos filas del DataFrame resultante
steam_games

,item_id,app_name,genres,year_of_release,publisher,developer
88310,761140,Lost Summoner Kitty,Action,2018,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Casual,2018,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Indie,2018,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Simulation,2018,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Strategy,2018,Kotoshiro,Kotoshiro
...,...,...,...,...,...,...
120442,610660,Russian Roads,Racing,2018,Laush Studio,Laush Dmitriy Sergeevich
120442,610660,Russian Roads,Simulation,2018,Laush Studio,Laush Dmitriy Sergeevich
120443,658870,EXIT 2 - Directions,Casual,2017,SIXNAILS,"xropi,stev3ns"
120443,658870,EXIT 2 - Directions,Indie,2017,SIXNAILS,"xropi,stev3ns"


In [66]:
steam_games["year_of_release"] = pd.to_datetime(steam_games["year_of_release"], errors='coerce').dt.year

# Reemplazar años inválidos (como 0) con NaN
steam_games["year_of_release"].replace({0: pd.NA}, inplace=True)

# Calcular la moda (año más común)
mode_year = steam_games["year_of_release"].mode()[0]

# Rellenar los valores NaN con la moda
steam_games["year_of_release"].fillna(mode_year, inplace=True)

In [67]:
steam_games

,item_id,app_name,genres,year_of_release,publisher,developer
88310,761140,Lost Summoner Kitty,Action,2018.0,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Casual,2018.0,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Indie,2018.0,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Simulation,2018.0,Kotoshiro,Kotoshiro
88310,761140,Lost Summoner Kitty,Strategy,2018.0,Kotoshiro,Kotoshiro
...,...,...,...,...,...,...
120442,610660,Russian Roads,Racing,2018.0,Laush Studio,Laush Dmitriy Sergeevich
120442,610660,Russian Roads,Simulation,2018.0,Laush Studio,Laush Dmitriy Sergeevich
120443,658870,EXIT 2 - Directions,Casual,2017.0,SIXNAILS,"xropi,stev3ns"
120443,658870,EXIT 2 - Directions,Indie,2017.0,SIXNAILS,"xropi,stev3ns"


**Carga del conjunto de datos transformados**

In [68]:
#Guardar la tabla en archivo CSV
# Definir el nombre del archivo y la ruta de guardado
ruta_games = 'data/steam_games_cleaned.csv'

# Guardar el DataFrame steam_games en el archivo CSV especificado
steam_games.to_csv(ruta_games, index=False, encoding='utf-8')
print(f'El archivo se ha almacenado exitosamente en: {ruta_games}')

El archivo se ha almacenado exitosamente en: data/steam_games_cleaned.csv


## **ETL de USER REVIEWS**

**Extracción de datos**

In [69]:
# Ruta al archivo de reseñas comprimido.
user_reviews_gz = "data/user_reviews.json.gz"
filas = []  # Lista para almacenar las reseñas.

# Abrir, leer y procesar el archivo gzip.
with gzip.open(user_reviews_gz, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():  
        filas.append(ast.literal_eval(line))  

# Convertir la lista de reseñas en un DataFrame de pandas.
user_reviews = pd.DataFrame(filas)

**Procesamiento de la Columna ‘reviews’ y Manejo de Valores Faltantes**
La columna ‘reviews’ de nuestro conjunto de datos tiene una estructura compleja, compuesta por una lista que almacena uno o más diccionarios como ítems. Realizaremos una serie de operaciones que nos permitirán reestructurar la información, dividirla en columnas separadas y asegurar la calidad de los datos.

Cada diccionario de la lista en la columna ‘reviews’ se transforma en una columna distinta.

In [70]:
#Revisamos la data
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Como podemos observar en la columna 'reviews' hay una lista y es necesario desanidarla. 

In [71]:
#Descomposición de la Columna ‘reviews’ en Subcolumnas
user_reviews2 = pd.json_normalize(user_reviews['reviews'])
                                                
#Unión de las Columnas ‘user_id’ y ‘user_url’ con las Subcolumnas
user_reviews2 = pd.concat([user_reviews[['user_id', 'user_url']], user_reviews2], axis=1)
                      
#Visualización de las Primeras Filas del Nuevo DataFrame
user_reviews2.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


**Reorganización de las Columnas en Registros Separados y Limpieza de los Datos** 
Conservamos las columnas ‘user_id’ y ‘user_url’ que identifican a cada usuario, y convertimos el resto de las columnas en filas individuales. Luego, descartamos las filas que tienen valores vacíos (‘None’) para evitar problemas de calidad de los datos.

In [72]:
# Transformamos las columnas en filas conservando 'user_id' y 'user_url' a través de pd.melt
user_reviews2 = pd.melt(user_reviews2, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')

# Quitamos las filas con valor None
user_reviews2 = user_reviews2.dropna()

In [73]:
#Revisamos la data
user_reviews2

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."
...,...,...,...,...
231919,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,8,"{'funny': '', 'posted': 'Posted August 15, 201..."
231921,76561198141079508,http://steamcommunity.com/profiles/76561198141...,8,"{'funny': '', 'posted': 'Posted August 2, 2014..."
232047,ShadowYT100,http://steamcommunity.com/id/ShadowYT100,8,"{'funny': '', 'posted': 'Posted July 31, 2015...."
232127,bestcustomurlevermade,http://steamcommunity.com/id/bestcustomurlever...,8,"{'funny': '', 'posted': 'Posted December 20, 2..."


In [74]:
# Convertir el diccionario en 'reviews' en columnas
user_reviews_expanded = pd.json_normalize(user_reviews2['reviews'])
# Renombrar las columnas con 'reviews' como prefijo
user_reviews_expanded.columns = ['reviews_' + col for col in user_reviews_expanded.columns]

In [75]:
#Revisamos las columnas
user_reviews_expanded.columns

Index(['reviews_funny', 'reviews_posted', 'reviews_last_edited',
       'reviews_item_id', 'reviews_helpful', 'reviews_recommend',
       'reviews_review'],
      dtype='object')

In [76]:
# Agregar las columnas 'user_id' y 'user_url' al principio
user_reviews_expanded = user_reviews2[['user_id', 'user_url']].join(user_reviews_expanded)

user_reviews = user_reviews_expanded

In [77]:
#Visualizar el dataframe
user_reviews.head(3)

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...


**Duplicados y Nulos.**

Primero, calculamos la cantidad de filas duplicadas en el DataFrame. Estos duplicados a menudo pueden surgir durante la recopilación o combinación de dato
Es importante identificarlos para no distorsionar nuestro análisis.

In [78]:
duplicados = user_reviews.duplicated().sum()
print(f"Se encontraron {duplicados} filas duplicadas.")

Se encontraron 12133 filas duplicadas.


In [79]:
# Eliminar los duplicados.
user_reviews.drop_duplicates(inplace=True, ignore_index=True)
user_reviews

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud
...,...,...,...,...,...,...,...,...,...
46898,76561198072632724,http://steamcommunity.com/profiles/76561198072...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46899,76561198080103160,http://steamcommunity.com/profiles/76561198080...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46900,spartaaagh,http://steamcommunity.com/id/spartaaagh,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46901,alexmr19,http://steamcommunity.com/id/alexmr19,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Añadimos la columna *'sentiment_analysis'*
Esta columna derivará su contenido de la evaluación del sentimiento natural expresado en las reseñas de los usuarios. La interpretación del sentimiento se basará en la siguiente escala numérica:

* 0: Indica una percepción negativa.
* 1: Se interpreta como un sentimiento neutro.
* 2: Representa una opinión positiva.

Para este procedimiento vamos a llevar a cabo la prueba de análisis de sentimiento utilizando la biblioteca `NLTK`

In [80]:
# Inicializa un objeto SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Define una función para asignar sentimientos
def assign_sentiment(text):
    if not isinstance(text, str):
        return 1  # Neutral para valores que no son cadenas de texto

    # Calcula la puntuación de sentimiento
    sentiment = sia.polarity_scores(text)
    
    # Clasifica el sentimiento basándose en la puntuación compuesta
    compound_score = sentiment['compound']
    if compound_score > 0.3:
        return 2  # Positivo
    elif compound_score < -0.3:
        return 0  # Negativo

    return 1  # Neutral

# Aplica la función a la columna 'reviews_review' usando una función lambda
user_reviews['sentiment_analysis'] = user_reviews['reviews_review'].apply(lambda x: assign_sentiment(x))

In [81]:
user_reviews.head(3)

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2


Esta nueva columna (sentiment_analysis) debe reemplazar la de reviews_review para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [82]:
user_reviews.drop('reviews_review', axis=1, inplace=True)

In [83]:
user_reviews.head(3)

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,2
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,2


**Procesamiento de Fechas en Reseñas de Usuarios**

In [84]:
def transform_date(date_str):
    # Check if date_str is a string
    if isinstance(date_str, str):
        try:
            # Convert the string to a date
            date = datetime.strptime(date_str, 'Posted %B %d, %Y.')
            # Format the date as 'YYYY-MM-DD'
            return date.strftime('%Y-%m-%d')
        except ValueError:
            # Handle or log the error if the date format is incorrect
            return None
    else:
        # Return None or some default value if date_str is not a string
        return None

In [85]:
# Aplica la función a la columna 'reviews_posted' en tu DataFrame user_reviews
user_reviews['reviews_posted'] = user_reviews['reviews_posted'].apply(transform_date)

# Visualiza el DataFrame resultante
user_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,2
2,evcentric,http://steamcommunity.com/id/evcentric,,None,,248820,No ratings yet,True,2
3,doctr,http://steamcommunity.com/id/doctr,,2013-10-14,,250320,2 of 2 people (100%) found this review helpful,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,2014-04-15,,211420,35 of 43 people (81%) found this review helpful,True,1


In [86]:
# Convertir la columna 'reviews_posted' al tipo datetim
user_reviews['reviews_posted'] = pd.to_datetime(user_reviews['reviews_posted'], errors='coerce')

# Extraer el año y crear la columna 'reviews_year', asignando -1 a los valores no válidos
user_reviews['reviews_year'] = user_reviews['reviews_posted'].dt.year.fillna(-1).astype(int)

# Reemplazar los valores -1 por "Dato no disponible" en la columna 'release_year'
user_reviews['reviews_year'] = user_reviews['reviews_year'].replace(-1, "Dato no disponible")

user_reviews

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis,reviews_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,2,2011
1,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,2,2014
2,evcentric,http://steamcommunity.com/id/evcentric,,NaT,,248820,No ratings yet,True,2,Dato no disponible
3,doctr,http://steamcommunity.com/id/doctr,,2013-10-14,,250320,2 of 2 people (100%) found this review helpful,True,2,2013
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,2014-04-15,,211420,35 of 43 people (81%) found this review helpful,True,1,2014
...,...,...,...,...,...,...,...,...,...,...
46898,76561198072632724,http://steamcommunity.com/profiles/76561198072...,NaN,NaT,NaN,NaN,NaN,NaN,1,Dato no disponible
46899,76561198080103160,http://steamcommunity.com/profiles/76561198080...,NaN,NaT,NaN,NaN,NaN,NaN,1,Dato no disponible
46900,spartaaagh,http://steamcommunity.com/id/spartaaagh,NaN,NaT,NaN,NaN,NaN,NaN,1,Dato no disponible
46901,alexmr19,http://steamcommunity.com/id/alexmr19,NaN,NaT,NaN,NaN,NaN,NaN,1,Dato no disponible


**Eliminación de Columnas Innecesaria**
Nos quedamos sólo con las columnas que necesitaremos para funciones y modelo de aprendizaje automático

In [87]:
#columnas innecesarias
columns_to_remove = ['reviews_last_edited', 'user_url', 'reviews_helpful','reviews_posted' ,'reviews_funny']
user_reviews = user_reviews.drop(columns_to_remove, axis=1)

#revisamos la tabla
user_reviews.head()

,user_id,reviews_item_id,reviews_recommend,sentiment_analysis,reviews_year
0,76561197970982479,1250,True,2,2011
1,js41637,251610,True,2,2014
2,evcentric,248820,True,2,Dato no disponible
3,doctr,250320,True,2,2013
4,maplemage,211420,True,1,2014


In [88]:
user_reviews.shape

(46903, 5)

**Exportar dataframe a archivo CSV**

In [89]:
#Guardar la tabla en archivo CSV
# Definir el nombre del archivo y la ruta de guardado
ruta_reviews = 'data/user_reviews_cleaned.csv'

# Guardar el DataFrame steam_games en el archivo CSV especificado
user_reviews.to_csv(ruta_reviews, index=False, encoding='utf-8')
print(f'El archivo se ha almacenado exitosamente en: {ruta_reviews}')

El archivo se ha almacenado exitosamente en: data/user_reviews_cleaned.csv


## **ETL de USER ITEMS**

In [90]:
# Ubicación del archivo comprimido JSON en el sistema de archivos
archivo_comprimido = 'data/users_items.json.gz'

# Inicializar lista para recolectar datos descomprimidos
datos_usuario = []

# Descomprimir y leer el archivo JSON línea por línea
with gzip.open(archivo_comprimido, 'rt', encoding='utf-8') as archivo:
    for linea in archivo.readlines():
        # Convertir cada línea de texto en un diccionario
        dato = ast.literal_eval(linea)
        # Agregar el diccionario a la lista de datos
        datos_usuario.append(dato)

# Convertir la lista de datos a un DataFrame para análisis
user_items = pd.DataFrame(datos_usuario)

In [91]:
#Revisemos las 5 primeras líneas
user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Para comenzar a trabajar con nuestro conjunto de datos, primero consultaremos la estructura de este utilizando el atributo shape para determinar el conteo de filas y columnas. Posteriormente, emplearemos las funciones head() y tail() para obtener una vista preliminar del conjunto y empezar a familiarizarnos con su contenido. El conjunto de datos que manejamos incluye varias columnas importantes:

**user_id:** Es el identificador único para cada usuario.
**items_count:** Refleja la cantidad de juegos que ha utilizado el usuario.
**steam_id:** Representa un identificador único del usuario dentro de la plataforma.
**user_url:** Corresponde a la URL del perfil del usuario.

Además, tenemos la columna items, que es una lista de diccionarios donde cada uno representa un juego específico que el usuario ha jugado. Esta columna se compone de varios subcampos relevantes:

- **item_id:** El identificador único del juego.
- **item_name:** El nombre del juego.
- **playtime_forever:** La cantidad total de tiempo que el usuario ha dedicado al juego.
- **playtime_2weeks:** El tiempo que el usuario ha jugado en las últimas dos semanas.

Estos campos son esenciales para realizar un análisis detallado del comportamiento de los usuarios en relación con los juegos que consumen.

In [92]:
# Selección de columnas específicas para retener en el análisis
columnas = ['user_id', 'items_count', 'steam_id']

# Conversión de la columna 'items' de formato JSON a una estructura tabular
user_items2 = pd.json_normalize(datos_usuario, record_path=['items'], meta=columnas)

# Reorganización de las columnas en el DataFrame resultante
user_items = user_items2[['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']]

# Muestra el DataFrame 'user_items' para revisión
user_items

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0
3,76561197970982479,277,40,Deathmatch Classic,0,0
4,76561197970982479,277,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...
5153204,76561198329548331,7,346330,BrainBread 2,0,0
5153205,76561198329548331,7,373330,All Is Dust,0,0
5153206,76561198329548331,7,388490,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,7,521570,You Have 10 Seconds 2,4,4


**Limpieza de Datos en el DataFrame de Items de Usuario**
Comenzamos identificando y eliminando cualquier fila duplicada. Esto es importante porque los duplicados pueden afectar la validez de cualquier análisis estadístico realizado en el conjunto de datos.

In [93]:
duplicados = user_items.duplicated().sum()
print(f"Filas duplicadas antes de la limpieza: {duplicados}")
user_items.drop_duplicates(inplace=True)

Filas duplicadas antes de la limpieza: 59104


C:\Users\USER\AppData\Local\Temp\ipykernel_11580\2577120648.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items.drop_duplicates(inplace=True)


In [94]:
#Revisamos si hay filas con valores nulos en el DataFrame.
nulos_por_columna = user_items.isnull().sum()
print("Valores nulos por columna antes de la limpieza:")
print(nulos_por_columna)

Valores nulos por columna antes de la limpieza:
user_id             0
items_count         0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64


**Guardar la data en archivo CSV**

In [95]:
# Definir el nombre del archivo y la ruta de guardado
ruta_items = 'data/user_items_cleaned.csv'

# Guardar el DataFrame steam_games en el archivo CSV especificado
user_items.to_csv(ruta_items, index=False, encoding='utf-8')
print(f'El archivo se ha almacenado exitosamente en: {ruta_items}')

El archivo se ha almacenado exitosamente en: data/user_items_cleaned.csv


**Guardar la data en archivo parquet**

In [96]:
# Definir el nombre del archivo y la ruta de guardado para el archivo Parquet
ruta_parquet = 'data/user_items_cleaned.parquet'

# Guardar el DataFrame user_items en el archivo Parquet especificado
user_items.to_parquet(ruta_parquet, index=False)

print(f'El archivo se ha almacenado exitosamente en: {ruta_parquet}')


El archivo se ha almacenado exitosamente en: data/user_items_cleaned.parquet
